In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Imports

# Creating Model
import os
import setuptools.dist
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from scipy.stats import multivariate_normal
import h5py

# Setting Environment
# os.environ['XLA_FLAGS'] = '--xla_gpu_cuda_data_dir=/opt/cuda'
print(tf.config.list_physical_devices('GPU'))
%matplotlib inline

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [5]:
# LOAD DATA

file_path = '/content/drive/MyDrive/ResearchFiles/RHEED_4848_test6.h5' # Change to your PATH

with h5py.File(file_path, 'r') as h5:
  spot = 'spot_2'
  g1 = np.array(h5['growth_1'][spot])
  g2 = np.array(h5['growth_2'][spot])
  g3 = np.array(h5['growth_3'][spot])
  g4 = np.array(h5['growth_4'][spot])
  g5 = np.array(h5['growth_5'][spot])
  g6 = np.array(h5['growth_6'][spot])
  g7 = np.array(h5['growth_7'][spot])
  g8 = np.array(h5['growth_8'][spot])
  g9 = np.array(h5['growth_9'][spot])
  g10 = np.array(h5['growth_10'][spot])
  g11 = np.array(h5['growth_11'][spot])
  g12 = np.array(h5['growth_12'][spot])
  images = np.concatenate((g1, g2, g3, g4, g5, g6, g7, g8, g9, g10, g11, g12))

arr = []
for index, image in enumerate(images):
  temp = image.astype(np.float32)
  image_max = np.max(image)
  temp /= image_max
  arr.append(temp)

images = np.array(arr)

dataset = tf.data.Dataset.from_tensor_slices(images)
batch_size = 1000
batch_num = 151
data_loader = dataset.shuffle(buffer_size=len(images)).repeat().batch(batch_size).take(batch_num)

images_array = np.load('/content/drive/MyDrive/ResearchFiles/images2.npy') # Change to your PATH
results_array = np.load('/content/drive/MyDrive/ResearchFiles/Results.npy') # Change to your PATH
print(images_array.shape)
print(results_array.shape)

output_scaler = StandardScaler()
data=results_array
output_scaler.fit(data)

(150985, 48, 48)
(150985, 5)


StandardScaler()

In [6]:
# Gaussian Function (TENSORFLOW)

print_example_guassian = 0

# mean_x, mean_y, cov_x, cov_y, theta
def generate_guassian(batch, image_shape):
    batch_size = batch.shape[0]
    mean_x, mean_y, cov_x, cov_y, theta = tf.unstack(batch, axis=-1)
    x = tf.range(image_shape[1], dtype=tf.float32)[:, tf.newaxis]  # Create a column vector
    x = tf.tile(x, [1, image_shape[0]])  # Repeat along columns

    y = tf.range(image_shape[0], dtype=tf.float32)[tf.newaxis, :]  # Create a row vector
    y = tf.tile(y, [image_shape[1], 1])  # Repeat along rows

    x = tf.tile(tf.expand_dims(x, 0), [batch_size, 1, 1])
    y = tf.tile(tf.expand_dims(y, 0), [batch_size, 1, 1])

    rota_matrix = tf.stack([tf.cos(theta), -tf.sin(theta), tf.sin(theta), tf.cos(theta)], axis=-1)
    rota_matrix = tf.reshape(rota_matrix, (batch_size, 2, 2))

    xy = tf.stack([x - tf.reshape(mean_x, (-1, 1, 1)), y - tf.reshape(mean_y, (-1, 1, 1))], axis=-1)
    xy = tf.einsum('bijk,bkl->bijl', xy, rota_matrix)

    img = tf.exp(-0.5 * (xy[:, :, :, 0]**2 / tf.reshape(cov_x, (-1, 1, 1))**2 + xy[:, :, :, 1]**2 / tf.reshape(cov_y, (-1, 1, 1))**2))

    return tf.expand_dims(img, axis=1)

if print_example_guassian:
    image_shape = (48, 48)
    batch = tf.convert_to_tensor([
        [19.2763, 24.8520, 11.2061,  6.8914,  0.7006]
        , [19.2763, 24.8520, 11.2061,  6.8914,  0.7006]
        , [19.2763, 24.8520, 11.2061,  6.8914,  50]
        , [19.2763, 24.8520, 11.2061,  6.8914,  0.7006]
        , [19.2763, 24.8520, 11.2061,  6.8914,  0.7006]
    ])

    generated_imgs = generate_guassian(batch, image_shape)
    plt.imshow(tf.squeeze(generated_imgs[2]))
    plt.show()

In [7]:
# Custom Loss Function (TENSORFLOW)

print_example_loss = 0

# @keras.saving.register_keras_serializable()
def custom_weighted_mse_loss(I, J, n):
  # Compute the weight
  W = tf.pow(I, n)

  # Compute the squared differences
  squared_diffs = tf.pow(I - J, 2)

  # Compute the weighted squared differences
  weighted_squared_diffs = W * squared_diffs

  # Compute the loss
  loss = tf.reduce_mean(weighted_squared_diffs)

  return loss

if print_example_loss:
  I = tf.random.normal((16, 1, 48, 48))
  J = tf.random.normal((16, 1, 48, 48))
  n = 2
  loss = custom_weighted_mse_loss(I, J, n)
  print("Custom Weighted MSE Loss:", loss.numpy())

In [8]:
# Model Architecture

model = tf.keras.Sequential(
    [
        layers.Conv2D(6, kernel_size=5, strides=1, padding='valid', data_format='channels_first')
        , layers.BatchNormalization(axis=1, momentum=0.1, epsilon=1e-05)
        , layers.ReLU()
        , layers.MaxPool2D(pool_size=4, strides=4)

        , layers.Conv2D(16, kernel_size=5, strides=1, padding='valid', data_format='channels_first')
        , layers.BatchNormalization(axis=1, momentum=0.1, epsilon=1e-05)
        , layers.ReLU()
        , layers.MaxPool2D(pool_size=2, strides=2)

        , layers.Flatten()
        , layers.Dense(98, activation='relu')
        , layers.Dense(52, activation='relu')
        , layers.Dense(5)
    ]
)

In [ ]:
# Training Loop
train_model = 1
save_model = 0
load_model = 0

if train_model:
    best_loss = float('inf')
    num_epochs = 200
    lr = 0.0001
    n = 1

    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

    model.compile(optimizer='adam', loss=custom_weighted_mse_loss)

    for epoch in range(num_epochs):
        running_loss = 0.0

        if epoch % 10 == 0:
            n += 0.1

        for images in tqdm(data_loader):
            images = tf.expand_dims(images, axis=1)
            with tf.GradientTape() as tape:
                embedding = model(images)
                unscaled_param = tf.constant(embedding * output_scaler.var_ ** 0.5 + output_scaler.mean_)
                final = generate_guassian(unscaled_param, (48,48))
                loss = custom_weighted_mse_loss(images, final, n)
            grads = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))

            running_loss += loss.numpy()
        average_loss = running_loss / len(data_loader)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {average_loss}")

if save_model:
    model.save("sequential.keras")
if load_model:
    model = keras.models.load_model('sequential.keras')

model.summary()